In [1]:
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob


In [2]:
df=pd.read_csv("Claeneddf.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35358 entries, 0 to 35357
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         35358 non-null  int64  
 1   helpful            16574 non-null  float64
 2   rating             31473 non-null  float64
 3   review_date        35093 non-null  object 
 4   site               34697 non-null  object 
 5   text               34869 non-null  object 
 6   verified_buyer     16145 non-null  object 
 7   verified_reviewer  29323 non-null  object 
 8   city               35358 non-null  object 
 9   state              35006 non-null  object 
 10  cleaned_text       34865 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 3.0+ MB


In [4]:
df.head()

,Unnamed: 0,helpful,rating,review_date,site,text,verified_buyer,verified_reviewer,city,state,cleaned_text
0,0,2.0,5.0,2019-05-26,BookIt.com,Just saved $600. on BookIt vs. another web sit...,VerifiedBuyer,VerifiedReviewer,Stuart of Springfield,IL,just saved 600 on bookit vs another web site h...
1,1,8.0,5.0,2019-03-28,BookIt.com,I had a great experience with BookIt. I needed...,VerifiedBuyer,VerifiedReviewer,Nichole of Roseville,CA,i had a great experience with bookit i needed ...
2,2,7.0,5.0,2019-02-10,BookIt.com,I have had nothing but courteous patient and w...,VerifiedBuyer,VerifiedReviewer,Cheryl of Corfu,NY,i have had nothing but courteous patient and w...
3,3,4.0,5.0,2019-05-22,BookIt.com,I just want to let you know that I had talked ...,VerifiedBuyer,VerifiedReviewer,Lisa of Flushing,MI,i just want to let you know that i had talked ...
4,4,4.0,4.0,2019-04-13,BookIt.com,I called to verify one of the All Inclusive Fe...,VerifiedBuyer,VerifiedReviewer,Brandon of Honolulu,HI,i called to verify one of the all inclusive fe...


In [12]:
# Handle NaN values in cleaned_text by replacing them with empty strings
df['cleaned_text'] = df['cleaned_text'].fillna("")


In [16]:

# Verify that there are no NaN values left
df['cleaned_text'].isna().sum()

0

In [20]:
 df['cleaned_text'].dtypes


dtype('O')

# Sentiment analysis

In [23]:
analyzer = SentimentIntensityAnalyzer()

In [25]:
# Function to get sentiment scores
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [27]:
df['sentiment'] = df['cleaned_text'].apply(get_sentiment)

In [29]:
sentiment_counts = df['sentiment'].value_counts()

In [33]:
sentiment_counts

sentiment
positive    17106
negative    16323
neutral      1929
Name: count, dtype: int64

In [35]:
df[['cleaned_text', 'sentiment']].head(5)

,cleaned_text,sentiment
0,just saved 600 on bookit vs another web site h...,positive
1,i had a great experience with bookit i needed ...,positive
2,i have had nothing but courteous patient and w...,positive
3,i just want to let you know that i had talked ...,positive
4,i called to verify one of the all inclusive fe...,negative


In [37]:
aspect_keywords = {
    'service': ['service', 'staff', 'customer', 'support', 'experience', 'help', 'care'],
    'food': ['food', 'meal', 'dining', 'restaurant', 'taste', 'menu', 'cuisine'],
    'amenities': ['room', 'facility', 'pool', 'wifi', 'cleanliness', 'comfort', 'parking']
}

In [39]:
def identify_aspects(text):
    aspects = []
    for aspect, keywords in aspect_keywords.items():
        if any(keyword in text for keyword in keywords):
            aspects.append(aspect)
    return aspects if aspects else ['general']

In [41]:
df['aspects'] = df['cleaned_text'].apply(identify_aspects)


In [45]:
aspect_df = df.explode('aspects')

In [47]:
aspect_df[['cleaned_text', 'aspects', 'sentiment']].head(5)

,cleaned_text,aspects,sentiment
0,just saved 600 on bookit vs another web site h...,general,positive
1,i had a great experience with bookit i needed ...,service,positive
2,i have had nothing but courteous patient and w...,service,positive
3,i just want to let you know that i had talked ...,general,positive
4,i called to verify one of the all inclusive fe...,service,negative


In [51]:
df['aspects'] = df['aspects'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))

In [57]:
output = "sentiment_analysis_results.csv"  
df.to_csv(output, index=False)

print(f" Data saved successfully: {output}")


 Data saved successfully: sentiment_analysis_results.csv
